<a href="https://colab.research.google.com/github/JarrydWannenburg/House_Prices_Adv_Reg/blob/main/0_House_Price_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Housing Prices - Advanced Regression Techniques

## Loading

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer, make_column_selector as Selector, TransformedTargetRegressor
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from lightgbm import LGBMRegressor
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_regression, chi2
from catboost import CatBoostRegressor

In [ ]:
# Edit default display options since there are lots of features
pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv('./data/train.csv')

# Uncomment when running script to submit predictions
#df_test = pd.read_csv('./data/test.csv')

# Fix anomaly in the real test data
# print(df_test.loc[df_test['YrSold'] < df_test['YearBuilt']][['YearBuilt', 'YrSold']])
# df_test.loc[1089, 'YrSold'] = 2008
# df_test.loc[1089, 'YearBuilt'] = 2007
# df_test.loc[1089, ['YearBuilt', 'YrSold']]

      YearBuilt  YrSold
1089       2008    2007


YearBuilt    2007
YrSold       2008
Name: 1089, dtype: object

In [ ]:
df.shape

(1460, 81)

## Data check

In [ ]:
# Check for duplicate variables
print('training duplicate columns:', df.duplicated().sum())
print('testing duplicate columns:', df_test.duplicated().sum())

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

## Pipeline

In [ ]:
dict1 = {"MSSubClass": str, 'MoSold':str}#, 'YrSold':str}
df = df.astype(dict1)
df_test = df_test.astype(dict1)

In [ ]:
class CategoricalTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, type_transformation_dict = {"MSSubClass": str,'MoSold':str}, 
                 excluded_cat_features=['PoolQC','Fence','MiscFeature', 'FireplaceQu']):
#         print('\nCat init \n')
        self.type_transformation_dict = type_transformation_dict
        self.excluded_cat_features = excluded_cat_features
        
    def fit(self, X, y=None):
#         print('\nCat fit \n')
        return self
    
    def transform(self, X, y=None):
#         print('\nCat trans \n')
        # Drop the excluded features (values with high null counts or useless)
        if len(self.excluded_cat_features) > 0:
            X.drop(columns=self.excluded_cat_features, inplace=True)
        # Transform types
        if len(self.type_transformation_dict) > 0:
            X.astype(self.type_transformation_dict)    

        X = X.replace([np.inf, -np.inf], np.nan)
        return X

In [ ]:
cat_pipe = Pipeline(steps=[
    ('transformer_function', CategoricalTransformer()),
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(drop='if_binary', handle_unknown='ignore')),
    ('selectKbest', SelectKBest(k=200))
])

In [ ]:
class NumericTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, right_skewed_cols=[], lower = 0.5, upper=10,
                excluded_num_features=['Id']):
#         print('\nNum init \n')
        self.right_skewed_cols = right_skewed_cols
        self.lower = lower
        self.upper = upper
        self.excluded_num_features = excluded_num_features
        
    def fit(self, X, y=None):
#         print('\nNum fit \n')
        return self
    
    def log_trans_right_skew(self, X, y=None):
#         print('\nNum log transform \n')
        # Get a list of columns with right skews
        self.right_skewed_cols = X.skew()[(X.skew() > lower) & (X.skew() < upper)].index.to_list()
        
        # Log transform right skew using log1p incase of 0 values
        for col in right_skewed_cols:
            X[col] = np.log1p(X[col])
            X.rename(columns={col:col+'_log'}, inplace=True)
        return X

    
    def transform(self, X, y=None):
        #print('\nNum trans \n')
        
        # Drop the excluded features (values with high null counts or useless)
        if len(self.excluded_num_features) > 0:
            X.drop(columns=self.excluded_num_features, inplace=True)
                   
        # Engineer new features and drop old ones to avoid multicollinearity
        X['bath_per_bed'] = (X['FullBath'] + 0.5*X['HalfBath'] + X['BsmtFullBath'] + X['BsmtHalfBath']) / X['BedroomAbvGr']
        #X['Bsmt_finished_prop'] = (X['BsmtFinSF1'] + X['BsmtFinSF2']) / X['TotalBsmtSF']
        X.drop(columns=['FullBath', 'HalfBath','BsmtFullBath','BsmtHalfBath'], inplace=True)
        
        # If data has right skew, log transform it to correct
        if len(self.right_skewed_cols) > 0:
            X = log_trans_right_skew(X)
            
        X = X.replace([np.inf, -np.inf], np.nan)
        return X

In [ ]:
num_pipe = Pipeline(steps=[
    ('transformer_function', NumericTransformer()),
    ('imputer', IterativeImputer()),
    ('scaler', StandardScaler()),
    ('pca', PCA(.85))
])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", cat_pipe, Selector(dtype_exclude=np.number)),
        ("num", num_pipe, Selector(dtype_include=np.number))
    ]
)

pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LinearRegression()) #objective='RMSE', random_seed=0
])

In [ ]:
X, y = df.drop(columns=['SalePrice']), df.loc[:, 'SalePrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=101)

# Uncomment when running script for submission
# X_train,y_train,X_test = df.drop(columns=['SalePrice']),df.loc[:, ['SalePrice']], df_test.copy()

In [ ]:
pipe.fit(X,y)

C:\Users\jarry\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('transformer_function',
                                                                   CategoricalTransformer()),
                                                                  ('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore')),
                                                                  ('selectKbest',
                                                                   SelectKBest(k=200))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000286941D9130>),
                                                 ('num',
                                                  Pipeline(steps=[('transformer_function',
                                                                   NumericTransformer()),
                                                                  ('imputer',
                                                                   IterativeImputer()),
                                                                  ('scaler',
                                                                   StandardScaler()),
                                                                  ('pca',
                                                                   PCA(n_components=0.85))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002869462C250>)])),
                ('model', LinearRegression())])

In [ ]:
# Estimate SalePrice in log form since heavily skewed
class CustomTargetTransformer(BaseEstimator, TransformerMixin):
  # no need to implement __init__ in this particular case
    def fit(self, target):
        return self

    def transform(self, target):
        print('\n*****************target_transform() called.\n')
        target = np.log1p(target)
        return target

    def inverse_transform(self, target):
        print('\n*****************inverse_target_transform() called.\n')
        target = np.expm1(target)
        return target


# create a TargetTransformer 
model = TransformedTargetRegressor(regressor=pipe, 
                                   transformer=CustomTargetTransformer(), 
                                   check_inverse=False)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

pred = pd.DataFrame(y_pred, columns=['SalePrice'])
pred['Id'] = [i+1461 for i in range(0,len(pred))]
pred = pred[['Id','SalePrice']]
pred.head()

pred.to_csv('./submissions/Pipeline_log.csv', index=False)


*****************target_transform() called.

Learning rate set to 0.041084
0:	learn: 0.3844605	total: 2.48ms	remaining: 2.48s
1:	learn: 0.3738481	total: 4.69ms	remaining: 2.34s
2:	learn: 0.3642585	total: 6.95ms	remaining: 2.31s
3:	learn: 0.3552165	total: 9.12ms	remaining: 2.27s
4:	learn: 0.3464818	total: 11.3ms	remaining: 2.24s
5:	learn: 0.3372943	total: 13.3ms	remaining: 2.2s
6:	learn: 0.3283921	total: 15.5ms	remaining: 2.19s
7:	learn: 0.3207374	total: 17.5ms	remaining: 2.17s
8:	learn: 0.3131639	total: 19.6ms	remaining: 2.15s
9:	learn: 0.3053076	total: 21.7ms	remaining: 2.15s
10:	learn: 0.2982987	total: 23.7ms	remaining: 2.13s
11:	learn: 0.2916551	total: 25.8ms	remaining: 2.13s
12:	learn: 0.2846776	total: 27.9ms	remaining: 2.12s
13:	learn: 0.2779774	total: 29.9ms	remaining: 2.1s
14:	learn: 0.2722554	total: 32.2ms	remaining: 2.11s
15:	learn: 0.2661866	total: 34.3ms	remaining: 2.11s
16:	learn: 0.2606456	total: 36.6ms	remaining: 2.12s
17:	learn: 0.2556197	total: 38.6ms	remaining: 2.1s
1

183:	learn: 0.0988189	total: 355ms	remaining: 1.57s
184:	learn: 0.0986309	total: 357ms	remaining: 1.57s
185:	learn: 0.0984628	total: 359ms	remaining: 1.57s
186:	learn: 0.0982758	total: 360ms	remaining: 1.57s
187:	learn: 0.0981914	total: 362ms	remaining: 1.56s
188:	learn: 0.0980499	total: 364ms	remaining: 1.56s
189:	learn: 0.0978662	total: 366ms	remaining: 1.56s
190:	learn: 0.0977425	total: 368ms	remaining: 1.56s
191:	learn: 0.0976925	total: 370ms	remaining: 1.56s
192:	learn: 0.0974199	total: 372ms	remaining: 1.55s
193:	learn: 0.0971286	total: 374ms	remaining: 1.55s
194:	learn: 0.0970338	total: 376ms	remaining: 1.55s
195:	learn: 0.0968784	total: 377ms	remaining: 1.55s
196:	learn: 0.0966403	total: 380ms	remaining: 1.55s
197:	learn: 0.0964966	total: 381ms	remaining: 1.54s
198:	learn: 0.0964362	total: 383ms	remaining: 1.54s
199:	learn: 0.0961661	total: 385ms	remaining: 1.54s
200:	learn: 0.0960314	total: 387ms	remaining: 1.54s
201:	learn: 0.0958955	total: 389ms	remaining: 1.54s
202:	learn: 

381:	learn: 0.0706560	total: 712ms	remaining: 1.15s
382:	learn: 0.0705185	total: 714ms	remaining: 1.15s
383:	learn: 0.0703847	total: 716ms	remaining: 1.15s
384:	learn: 0.0703199	total: 718ms	remaining: 1.15s
385:	learn: 0.0701114	total: 720ms	remaining: 1.14s
386:	learn: 0.0700904	total: 722ms	remaining: 1.14s
387:	learn: 0.0699465	total: 723ms	remaining: 1.14s
388:	learn: 0.0697835	total: 725ms	remaining: 1.14s
389:	learn: 0.0696959	total: 727ms	remaining: 1.14s
390:	learn: 0.0696493	total: 729ms	remaining: 1.14s
391:	learn: 0.0695953	total: 731ms	remaining: 1.13s
392:	learn: 0.0694983	total: 733ms	remaining: 1.13s
393:	learn: 0.0694079	total: 735ms	remaining: 1.13s
394:	learn: 0.0692664	total: 736ms	remaining: 1.13s
395:	learn: 0.0692513	total: 738ms	remaining: 1.13s
396:	learn: 0.0691269	total: 740ms	remaining: 1.12s
397:	learn: 0.0690386	total: 742ms	remaining: 1.12s
398:	learn: 0.0688571	total: 743ms	remaining: 1.12s
399:	learn: 0.0686865	total: 745ms	remaining: 1.12s
400:	learn: 

582:	learn: 0.0524821	total: 1.07s	remaining: 766ms
583:	learn: 0.0524240	total: 1.07s	remaining: 765ms
584:	learn: 0.0523207	total: 1.07s	remaining: 763ms
585:	learn: 0.0522169	total: 1.08s	remaining: 761ms
586:	learn: 0.0521517	total: 1.08s	remaining: 759ms
587:	learn: 0.0520194	total: 1.08s	remaining: 757ms
588:	learn: 0.0518814	total: 1.08s	remaining: 755ms
589:	learn: 0.0517773	total: 1.08s	remaining: 753ms
590:	learn: 0.0516797	total: 1.09s	remaining: 752ms
591:	learn: 0.0515937	total: 1.09s	remaining: 750ms
592:	learn: 0.0514827	total: 1.09s	remaining: 748ms
593:	learn: 0.0514321	total: 1.09s	remaining: 746ms
594:	learn: 0.0513129	total: 1.09s	remaining: 744ms
595:	learn: 0.0512229	total: 1.09s	remaining: 742ms
596:	learn: 0.0512137	total: 1.1s	remaining: 741ms
597:	learn: 0.0511298	total: 1.1s	remaining: 739ms
598:	learn: 0.0511109	total: 1.1s	remaining: 737ms
599:	learn: 0.0510224	total: 1.1s	remaining: 735ms
600:	learn: 0.0509091	total: 1.1s	remaining: 733ms
601:	learn: 0.050

776:	learn: 0.0394608	total: 1.43s	remaining: 411ms
777:	learn: 0.0393974	total: 1.43s	remaining: 409ms
778:	learn: 0.0393093	total: 1.43s	remaining: 407ms
779:	learn: 0.0392172	total: 1.44s	remaining: 405ms
780:	learn: 0.0391546	total: 1.45s	remaining: 408ms
781:	learn: 0.0391130	total: 1.46s	remaining: 406ms
782:	learn: 0.0390314	total: 1.46s	remaining: 404ms
783:	learn: 0.0389814	total: 1.46s	remaining: 402ms
784:	learn: 0.0389208	total: 1.46s	remaining: 400ms
785:	learn: 0.0388545	total: 1.46s	remaining: 398ms
786:	learn: 0.0387889	total: 1.46s	remaining: 396ms
787:	learn: 0.0387332	total: 1.47s	remaining: 394ms
788:	learn: 0.0386321	total: 1.47s	remaining: 393ms
789:	learn: 0.0385864	total: 1.47s	remaining: 391ms
790:	learn: 0.0384861	total: 1.47s	remaining: 389ms
791:	learn: 0.0384396	total: 1.47s	remaining: 387ms
792:	learn: 0.0383735	total: 1.48s	remaining: 385ms
793:	learn: 0.0383679	total: 1.48s	remaining: 383ms
794:	learn: 0.0383082	total: 1.48s	remaining: 381ms
795:	learn: 

969:	learn: 0.0299181	total: 1.79s	remaining: 55.4ms
970:	learn: 0.0299045	total: 1.79s	remaining: 53.6ms
971:	learn: 0.0298278	total: 1.79s	remaining: 51.7ms
972:	learn: 0.0297967	total: 1.8s	remaining: 49.9ms
973:	learn: 0.0297561	total: 1.8s	remaining: 48ms
974:	learn: 0.0296980	total: 1.8s	remaining: 46.2ms
975:	learn: 0.0296658	total: 1.8s	remaining: 44.3ms
976:	learn: 0.0296182	total: 1.8s	remaining: 42.5ms
977:	learn: 0.0295681	total: 1.8s	remaining: 40.6ms
978:	learn: 0.0295174	total: 1.81s	remaining: 38.8ms
979:	learn: 0.0294719	total: 1.81s	remaining: 36.9ms
980:	learn: 0.0294402	total: 1.81s	remaining: 35.1ms
981:	learn: 0.0293850	total: 1.81s	remaining: 33.2ms
982:	learn: 0.0293420	total: 1.81s	remaining: 31.4ms
983:	learn: 0.0292933	total: 1.82s	remaining: 29.5ms
984:	learn: 0.0292478	total: 1.82s	remaining: 27.7ms
985:	learn: 0.0292141	total: 1.82s	remaining: 25.8ms
986:	learn: 0.0291598	total: 1.82s	remaining: 24ms
987:	learn: 0.0291524	total: 1.82s	remaining: 22.2ms
988

In [ ]:
mse = mean_squared_error(y_test, y_pred)
np.sqrt(mse)

27363.889117541577

## Hyper Param Tuning

In [ ]:
from catboost import CatBoostRegressor

class ClfSwitcher(BaseEstimator):
    def __init__(
        self, 
        estimator = LinearRegression()):
        """
        A Custom BaseEstimator that can switch between classifiers.
        :param estimator: sklearn object - The regressor/classifier
        """ 

        self.estimator = estimator


    def fit(self, X, y=None, **kwargs):
        self.estimator.fit(X, y)
        return self


    def predict(self, X, y=None):
        return self.estimator.predict(X)


    def predict_proba(self, X):
        return self.estimator.predict_proba(X)


    def score(self, X, y):
        return self.estimator.score(X, y)

### SelectKbest

In [ ]:
hyper_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

full_pipe_clf = Pipeline([
    ('pipe', hyper_pipe),
    ('clf', ClfSwitcher())
])

parameters = [
    {
        'pipe__preprocessor__cat__selectKbest__k': [200],
        'pipe__preprocessor__num__pca__n_components': [0.85],
        'clf__estimator': [CatBoostRegressor(objective='RMSE')],
        #'clf__estimator__learning_rate': [0.01,0.015,0.02],
    }
]

import warnings
warnings.filterwarnings('ignore')

gscv = GridSearchCV(full_pipe_clf, parameters, cv=10)
gscv.fit(X, y)

Learning rate set to 0.042748
0:	learn: 78598.3695870	total: 2.4ms	remaining: 2.4s
1:	learn: 76477.2510250	total: 4.72ms	remaining: 2.35s
2:	learn: 74482.6765007	total: 7ms	remaining: 2.33s
3:	learn: 72562.0626991	total: 9.15ms	remaining: 2.28s
4:	learn: 70662.6692461	total: 11.3ms	remaining: 2.26s
5:	learn: 68776.7573099	total: 13.5ms	remaining: 2.23s
6:	learn: 67006.8046057	total: 15.5ms	remaining: 2.2s
7:	learn: 65562.1054582	total: 17.6ms	remaining: 2.18s
8:	learn: 63927.7106597	total: 20ms	remaining: 2.2s
9:	learn: 62275.7880923	total: 22.2ms	remaining: 2.2s
10:	learn: 60644.1049883	total: 24.5ms	remaining: 2.2s
11:	learn: 59232.1679982	total: 26.5ms	remaining: 2.18s
12:	learn: 57810.1206829	total: 28.8ms	remaining: 2.18s
13:	learn: 56530.8136190	total: 30.9ms	remaining: 2.17s
14:	learn: 55237.5556449	total: 32.9ms	remaining: 2.16s
15:	learn: 53949.9947409	total: 35ms	remaining: 2.15s
16:	learn: 52819.2492408	total: 37.3ms	remaining: 2.15s
17:	learn: 51746.0554296	total: 39.6ms	re

184:	learn: 18077.8827972	total: 357ms	remaining: 1.57s
185:	learn: 18045.1057143	total: 359ms	remaining: 1.57s
186:	learn: 17994.5017224	total: 361ms	remaining: 1.57s
187:	learn: 17957.1883807	total: 363ms	remaining: 1.57s
188:	learn: 17931.3531133	total: 364ms	remaining: 1.56s
189:	learn: 17873.2845023	total: 369ms	remaining: 1.57s
190:	learn: 17839.5388949	total: 372ms	remaining: 1.57s
191:	learn: 17792.4843562	total: 374ms	remaining: 1.57s
192:	learn: 17737.7684574	total: 377ms	remaining: 1.57s
193:	learn: 17696.4296979	total: 379ms	remaining: 1.57s
194:	learn: 17660.9247924	total: 381ms	remaining: 1.57s
195:	learn: 17619.3834625	total: 383ms	remaining: 1.57s
196:	learn: 17585.6583635	total: 385ms	remaining: 1.57s
197:	learn: 17542.7875152	total: 387ms	remaining: 1.57s
198:	learn: 17506.1202764	total: 389ms	remaining: 1.57s
199:	learn: 17483.1909985	total: 391ms	remaining: 1.56s
200:	learn: 17463.9811908	total: 393ms	remaining: 1.56s
201:	learn: 17429.3853282	total: 395ms	remaining

374:	learn: 12631.8076784	total: 716ms	remaining: 1.19s
375:	learn: 12589.5832565	total: 718ms	remaining: 1.19s
376:	learn: 12568.4208240	total: 720ms	remaining: 1.19s
377:	learn: 12546.6972441	total: 722ms	remaining: 1.19s
378:	learn: 12537.3153279	total: 724ms	remaining: 1.19s
379:	learn: 12511.4513842	total: 725ms	remaining: 1.18s
380:	learn: 12476.8614732	total: 727ms	remaining: 1.18s
381:	learn: 12457.6090425	total: 729ms	remaining: 1.18s
382:	learn: 12450.4856148	total: 731ms	remaining: 1.18s
383:	learn: 12419.6196503	total: 733ms	remaining: 1.18s
384:	learn: 12409.8927688	total: 735ms	remaining: 1.17s
385:	learn: 12397.4303901	total: 737ms	remaining: 1.17s
386:	learn: 12380.1477475	total: 739ms	remaining: 1.17s
387:	learn: 12370.6693239	total: 740ms	remaining: 1.17s
388:	learn: 12358.3339143	total: 742ms	remaining: 1.17s
389:	learn: 12335.5056759	total: 744ms	remaining: 1.16s
390:	learn: 12323.3767332	total: 746ms	remaining: 1.16s
391:	learn: 12309.2607433	total: 748ms	remaining

570:	learn: 9311.1689149	total: 1.08s	remaining: 809ms
571:	learn: 9305.0100924	total: 1.08s	remaining: 807ms
572:	learn: 9288.7519621	total: 1.08s	remaining: 805ms
573:	learn: 9269.2252904	total: 1.08s	remaining: 804ms
574:	learn: 9261.8068677	total: 1.08s	remaining: 802ms
575:	learn: 9244.4000248	total: 1.09s	remaining: 800ms
576:	learn: 9236.6330858	total: 1.09s	remaining: 798ms
577:	learn: 9226.0714473	total: 1.09s	remaining: 796ms
578:	learn: 9215.3144306	total: 1.09s	remaining: 794ms
579:	learn: 9194.2453024	total: 1.09s	remaining: 792ms
580:	learn: 9184.5503957	total: 1.09s	remaining: 790ms
581:	learn: 9167.6870933	total: 1.1s	remaining: 788ms
582:	learn: 9160.6565672	total: 1.1s	remaining: 786ms
583:	learn: 9146.6920896	total: 1.11s	remaining: 793ms
584:	learn: 9137.2850584	total: 1.11s	remaining: 791ms
585:	learn: 9132.3233255	total: 1.12s	remaining: 789ms
586:	learn: 9120.1476893	total: 1.12s	remaining: 787ms
587:	learn: 9104.6224277	total: 1.12s	remaining: 785ms
588:	learn: 

755:	learn: 7158.3614232	total: 1.44s	remaining: 464ms
756:	learn: 7155.5533342	total: 1.44s	remaining: 462ms
757:	learn: 7143.9514461	total: 1.44s	remaining: 460ms
758:	learn: 7134.4643335	total: 1.44s	remaining: 458ms
759:	learn: 7120.9626531	total: 1.44s	remaining: 456ms
760:	learn: 7116.9620096	total: 1.45s	remaining: 454ms
761:	learn: 7108.9332997	total: 1.45s	remaining: 452ms
762:	learn: 7098.6107306	total: 1.45s	remaining: 451ms
763:	learn: 7090.6371887	total: 1.45s	remaining: 449ms
764:	learn: 7078.1653910	total: 1.45s	remaining: 447ms
765:	learn: 7062.0194426	total: 1.46s	remaining: 445ms
766:	learn: 7049.5819832	total: 1.46s	remaining: 443ms
767:	learn: 7042.2472242	total: 1.46s	remaining: 441ms
768:	learn: 7030.0007302	total: 1.46s	remaining: 439ms
769:	learn: 7021.8026135	total: 1.46s	remaining: 437ms
770:	learn: 7015.7383599	total: 1.47s	remaining: 435ms
771:	learn: 7007.6975636	total: 1.47s	remaining: 433ms
772:	learn: 6993.0264381	total: 1.47s	remaining: 431ms
773:	learn

...

368:	learn: 12841.3328593	total: 717ms	remaining: 1.23s
369:	learn: 12814.2927775	total: 719ms	remaining: 1.22s
370:	learn: 12787.3666553	total: 721ms	remaining: 1.22s
371:	learn: 12766.5847433	total: 723ms	remaining: 1.22s
372:	learn: 12745.9540385	total: 725ms	remaining: 1.22s
373:	learn: 12722.6303248	total: 727ms	remaining: 1.22s
374:	learn: 12709.5754698	total: 728ms	remaining: 1.21s
375:	learn: 12688.0868080	total: 730ms	remaining: 1.21s
376:	learn: 12673.2314078	total: 732ms	remaining: 1.21s
377:	learn: 12655.0883558	total: 734ms	remaining: 1.21s
378:	learn: 12650.1466129	total: 736ms	remaining: 1.21s
379:	learn: 12636.6108009	total: 738ms	remaining: 1.2s
380:	learn: 12621.3076868	total: 740ms	remaining: 1.2s
381:	learn: 12594.0403916	total: 742ms	remaining: 1.2s
382:	learn: 12581.9565652	total: 743ms	remaining: 1.2s
383:	learn: 12564.5978257	total: 745ms	remaining: 1.2s
384:	learn: 12547.5502962	total: 747ms	remaining: 1.19s
385:	learn: 12519.8938564	total: 749ms	remaining: 1.1

565:	learn: 9725.8936254	total: 1.08s	remaining: 826ms
566:	learn: 9714.2589955	total: 1.08s	remaining: 824ms
567:	learn: 9708.2236873	total: 1.08s	remaining: 822ms
568:	learn: 9693.8688735	total: 1.08s	remaining: 820ms
569:	learn: 9687.5114027	total: 1.08s	remaining: 818ms
570:	learn: 9668.2289787	total: 1.08s	remaining: 816ms
571:	learn: 9649.8508605	total: 1.09s	remaining: 814ms
572:	learn: 9646.2788366	total: 1.09s	remaining: 812ms
573:	learn: 9625.5198046	total: 1.09s	remaining: 810ms
574:	learn: 9613.0487079	total: 1.09s	remaining: 808ms
575:	learn: 9607.6428358	total: 1.09s	remaining: 806ms
576:	learn: 9594.2427156	total: 1.1s	remaining: 804ms
577:	learn: 9583.4006731	total: 1.1s	remaining: 802ms
578:	learn: 9576.5522256	total: 1.1s	remaining: 801ms
579:	learn: 9566.7915622	total: 1.1s	remaining: 799ms
580:	learn: 9555.1605549	total: 1.1s	remaining: 797ms
581:	learn: 9533.9914374	total: 1.11s	remaining: 795ms
582:	learn: 9520.0203984	total: 1.11s	remaining: 793ms
583:	learn: 950

761:	learn: 7648.7841802	total: 1.44s	remaining: 449ms
762:	learn: 7638.5451466	total: 1.44s	remaining: 447ms
763:	learn: 7633.6224102	total: 1.44s	remaining: 445ms
764:	learn: 7628.5332474	total: 1.44s	remaining: 443ms
765:	learn: 7621.2103114	total: 1.44s	remaining: 441ms
766:	learn: 7610.0846877	total: 1.45s	remaining: 439ms
767:	learn: 7601.5284647	total: 1.45s	remaining: 438ms
768:	learn: 7588.2899282	total: 1.45s	remaining: 436ms
769:	learn: 7581.5067024	total: 1.45s	remaining: 434ms
770:	learn: 7573.7336827	total: 1.45s	remaining: 432ms
771:	learn: 7562.7009587	total: 1.46s	remaining: 430ms
772:	learn: 7551.0308127	total: 1.46s	remaining: 428ms
773:	learn: 7547.2498890	total: 1.46s	remaining: 426ms
774:	learn: 7541.4399525	total: 1.46s	remaining: 424ms
775:	learn: 7536.6808575	total: 1.46s	remaining: 422ms
776:	learn: 7530.5215600	total: 1.46s	remaining: 420ms
777:	learn: 7518.0122887	total: 1.47s	remaining: 419ms
778:	learn: 7510.0950889	total: 1.47s	remaining: 417ms
779:	learn

959:	learn: 6143.9533687	total: 1.8s	remaining: 74.9ms
960:	learn: 6139.0440350	total: 1.8s	remaining: 73ms
961:	learn: 6128.3936374	total: 1.8s	remaining: 71.2ms
962:	learn: 6117.0583058	total: 1.8s	remaining: 69.3ms
963:	learn: 6109.7577503	total: 1.8s	remaining: 67.4ms
964:	learn: 6103.1071035	total: 1.81s	remaining: 65.6ms
965:	learn: 6100.0066759	total: 1.81s	remaining: 63.7ms
966:	learn: 6098.5524311	total: 1.81s	remaining: 61.8ms
967:	learn: 6087.5745514	total: 1.81s	remaining: 59.9ms
968:	learn: 6081.6365395	total: 1.81s	remaining: 58.1ms
969:	learn: 6079.8397261	total: 1.82s	remaining: 56.2ms
970:	learn: 6076.6153836	total: 1.82s	remaining: 54.3ms
971:	learn: 6069.3967866	total: 1.82s	remaining: 52.5ms
972:	learn: 6059.6135679	total: 1.82s	remaining: 50.6ms
973:	learn: 6059.0951059	total: 1.82s	remaining: 48.7ms
974:	learn: 6052.8819038	total: 1.83s	remaining: 46.8ms
975:	learn: 6047.5852914	total: 1.83s	remaining: 45ms
976:	learn: 6042.0078030	total: 1.83s	remaining: 43.1ms
9

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('pipe',
                                        Pipeline(steps=[('preprocessor',
                                                         ColumnTransformer(transformers=[('cat',
                                                                                          Pipeline(steps=[('transformer_function',
                                                                                                           CategoricalTransformer()),
                                                                                                          ('imputer',
                                                                                                           SimpleImputer(strategy='most_frequent')),
                                                                                                          ('encoder',
                                                                                                           OneHotEncoder(drop='if_binary',
                                                                                                                         handle_unknown='ignore')),
                                                                                                          ('selectKbest',
                                                                                                           SelectKBest(k=200))]),
                                                                                          <sk...
                                                                                                           StandardScaler()),
                                                                                                          ('pca',
                                                                                                           PCA(n_components=0.85))]),
                                                                                          <sklearn.compose._column_transformer.make_column_selector object at 0x0000015B7EB2C700>)]))])),
                                       ('clf', ClfSwitcher())]),
             param_grid=[{'clf__estimator': [<catboost.core.CatBoostRegressor object at 0x0000015B7E14F130>],
                          'pipe__preprocessor__cat__selectKbest__k': [200],
                          'pipe__preprocessor__num__pca__n_components': [0.85]}])

In [ ]:
gs = pd.DataFrame(gscv.cv_results_)
# Display the top ranking default models
display(gs.sort_values(by='rank_test_score', ascending=True))

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__estimator,param_pipe__preprocessor__cat__selectKbest__k,param_pipe__preprocessor__num__pca__n_components,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,2.286331,0.046238,0.01495,0.000002,<catboost.core.CatBoostRegressor object at 0x0...,200,0.85,{'clf__estimator': <catboost.core.CatBoostRegr...,0.922741,0.905083,0.936919,0.864517,0.887816,0.898914,0.905562,0.916477,0.821915,0.902645,0.896259,0.030954,1


In [ ]:
gs.params[0]

{'clf__estimator': <catboost.core.CatBoostRegressor at 0x15b7e14f130>,
 'pipe__preprocessor__cat__selectKbest__k': 200,
 'pipe__preprocessor__num__pca__n_components': 0.85}

In [ ]:
best = gscv.best_estimator_
y_pred = best.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
round(np.sqrt(mse),4)

5576.6415

cat_pca85_k200: 5208.2218
- Bath per bed, drop ['PoolQC','Fence','MiscFeature','FireplaceQu']

cat_pca85_k200: 5576.6415
- Bath per bed, summer_sale, drop ['PoolQC','Fence','MiscFeature','FireplaceQu']

cat_pca75_k201: 5545.9846

In [ ]:
pred = pd.DataFrame(y_pred, columns=['SalePrice'])
pred['Id'] = [i+1461 for i in range(0,len(pred))]
pred = pred[['Id','SalePrice']]
pred.head()

pred.to_csv('./submissions/Pipeline.csv', index=False)

### Base CatBoost

### Tuning CatBoost

# Other Code

### Model Selection

### Ensemble model

### Model selection gridsearchcv